In [1]:
from utils import calculate_auc
from data_processor import BoxingProcessor
from train_test_split_games import BoxingSplitter
from true_skill_through_time import TrueSkillThroughTimeApplied

%load_ext autoreload
%autoreload 2

# Run this to generate and save the data

In [2]:
# boxing_instance = BoxingProcessor("data/boxer_wiki_urls.txt")
# boxing_matches_df = boxing_instance.pull_data()
# boxing_matches_cleaned_df = boxing_instance.process_games_data(boxing_matches_df)
# boxing_matches_cleaned_df = boxing_matches_cleaned_df.reset_index()
# boxing_matches_cleaned_df = boxing_matches_cleaned_df.rename(columns={'index': 'game_index'})
# boxing_matches_cleaned_df.to_parquet('data/boxing_matches_cleaned.parquet')

# Split the data into train and test sets

In [3]:
boxing_splitter = BoxingSplitter("data/boxing_matches_cleaned.parquet", "data/players_ge_40_matches_lst_boxing.json")

In [4]:
boxing_train, boxing_test = boxing_splitter.train_test_split()

100%|███████████████████████████████████████████████████████████████████████████████| 303/303 [00:00<00:00, 362.49it/s]


In [5]:
boxing_train.shape, boxing_test.shape

((16889, 5), (4116, 5))

# Tune and Apply the TrueSkill Through Time

## Only run the below if you'd like to re-tune the hyperparameters

In [6]:
trueskill = TrueSkillThroughTimeApplied(boxing_train)

In [7]:
# trueskill.learn_optimal_parameters(game_type='boxing')

## Set the hyperparameters

In [8]:
# MLE: 5786.7752
optimal_gamma = 0.0738
optimal_sigma = 0.6004
optimal_beta = 0.2843
trueskill.set_optimal_parameters(gamma = optimal_gamma, sigma = optimal_sigma, beta = optimal_beta)

In [9]:
skill_curves = trueskill.set_skill_curves()

In [19]:
top_boxers_online = ['Muhammad Ali', 
                     'Joe Louis', 
                     'Sugar Ray Robinson', 
                     'Rocky Marciano', 
                     'Floyd Mayweather Jr', 
                     'Manny Pacquiao', 
                     'Jack Dempsey', 
                     'Roberto Durán', 
                     'Henry Armstrong', 
                     'Willie Pep', 
                     'Mike Tyson']

In [20]:
trueskill.plot_player_skills(players = top_boxers_online, width=1000, height=500, burnin=10, x_axis_format="%Y", y_lims=[-0, 3], game='boxing')

alt.LayerChart(...)

In [14]:
trueskill.plot_calibration()

C:\Users\Duane\Dropbox\consulting\website\public_repo\true_theta\posts\trueskill\true_skill_through_time.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_means = df.groupby('win_prob_bucket').agg(


alt.LayerChart(...)

In [17]:
trueskill.plot_calibration_oos(oos_data=boxing_test, game='Boxing')

C:\Users\Duane\Dropbox\consulting\website\public_repo\true_theta\posts\trueskill\true_skill_through_time.py:330: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  bucket_means = df.groupby("win_prob_bucket").agg(


alt.LayerChart(...)

In [16]:
calculate_auc(boxing_test, skill_curves, optimal_beta)

np.float64(0.7499028650869048)